# Modified HadUK-Grid from planar to geodesic coordenates - NEW VERSION
<div class="alert alert-block alert-warning">
 <p style="color:black"> <b>Caution: This should only be run BEFORE creating the indices datasets and AFTER making sure there are not values under 1mm in the rainfall variable</b></p>
</div>

This process is as follows:

* All HadUK-Grid files are merge into one nc file - rainfall_hadukgrid_uk_5km_day_2001-2019.nc
* The following variables need to be removed using NCO:
    - `ncks -C -O -x -v projection_y_coordinate_bnds,projection_x_coordinate_bnds rainfall_hadukgrid_uk_5km_day_2001-2019.nc rainfall_hadukgrid_uk_5km_day_2001-2019_step1.nc`
    - `ncks -C -x -v latitude,longitude,time_bnds rainfall_hadukgrid_uk_5km_day_2001-2019_step1.nc rainfall_hadukgrid_uk_5km_day_OSGB36_2001-2019.nc`
    
## Reproject data using rioxarray

In [ ]:
import rioxarray
from pyproj import CRS

# Set directory to read and for outputs
fldr_src = Path('/mnt/d/MRes_dataset/search_data/haduk_cedac_uk_xclim/')
fldr_out = Path('/mnt/d/MRes_dataset/active_data/106_precp')

# Create list with files
fls_lst = fldr_src.glob('**/*.nc')

# Read HadUK-Grid data and merge all into one file
dataset_HADUK = xr.open_mfdataset(paths=fls_lst, combine='by_coords', parallel=True)
dataset_HADUK.to_netcdf(Path(fldr_out / 'rainfall_hadukgrid_uk_5km_day_2001-2019.nc'))

print('---------')
print('Use NCO to modify the rainfall_hadukgrid_uk_5km_day_2001-2019.nc file before processiding to next step')
print('---------')
print('File saved')

## Transforming the dataset using rioxarray

In [ ]:
import rioxarray
from pyproj import CRS

# # Reading data with the CRS information 
xds = xr.open_dataset(Path(fldr_out / 'rainfall_hadukgrid_uk_5km_day_OSGB36_2001-2019.nc'))

# # Read the CRS information from the transver_mercator variable
xds.rio.set_spatial_dims(x_dim="projection_x_coordinate", y_dim="projection_y_coordinate", inplace=True)
xds.rio.write_crs("epsg:27700", inplace=True)

# Transform the data
xds_lonlat = xds.rio.reproject("EPSG:4326")

# Remove grid_mapping before saving as it causes an issue
xds_lonlat.rainfall.attrs.pop('grid_mapping')

# # Save data in WGS84
xds_lonlat.to_netcdf(Path(fldr_out/ 'rainfall_hadukgrid_uk_5km_day_WGS1984_2001-2019.nc'))

print('---------')
print('The data has been transformed')
print('---------')

## Setting transformed dataset for Xclim

<div class="alert alert-block alert-warning">
 <p style="color:black"> <b>Caution: Before running xclim the transformed data attribute need changing, so coordinates show as latitude and longitude</b></p>
</div>

In [ ]:
# Read transformed file
dsin = Dataset(Path(fldr_out/ 'rainfall_hadukgrid_uk_5km_day_WGS84_2001-2019.nc'))

# Output file
dsout = Dataset(Path(fldr_out / 'rainfall_hadukgrid_uk_5km_day_WGS84LatLon_2001-2019.nc'), mode='w',format='NETCDF3_CLASSIC')

# Create dimensions
for dimname,dim in f.dimensions.items():
    dsout.createDimension(dimname,len(dim))

# Copy variables
# transform_mercator is dropped as it is needed.
for v_name, varin in dsin.variables.items():
    if v_name == 'x':
        outVar = dsout.createVariable('longitude', varin.datatype, ('longitude',))
        # Copy variable attributes
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
        outVar[:] = varin[:]
    if v_name == 'y':
        outVar = dsout.createVariable('latitude', varin.datatype, ('latitude',))
        # Copy variable attributes
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
        outVar[:] = varin[:]
    if v_name == 'rainfall':
        outVar = dsout.createVariable(v_name, varin.datatype, ('time', 'latitude', 'longitude'))
        # Copy variable attributes
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
        outVar[:] = varin[:]
    if v_name == 'time':
        outVar = dsout.createVariable(v_name, np.float64, varin.dimensions)
        # Copy variable attributes
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
        outVar[:] = varin[:]

# close the output file to
dsout.close()

print('---------')
print('The transformed data is already to go')
print('---------')